### Parallel Chain
Process these in parallel: Prompt1 (Notes) Prompt2 (Quiz)
Merge these into a single prompt: Prompt3
Invoke this merged prompt

In [4]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv 
from langchain_core.prompts import PromptTemplate
import os 

os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")
os.environ["HF_HOME"] = os.getenv("HF_HOME")

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation"
)
model1 = ChatHuggingFace(llm=llm)
model2 = ChatHuggingFace(llm=llm)


In [5]:
text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

prompt1 = PromptTemplate(
    template="Generate short and simple notes for the following: {text} \n",
    input_variables=["text"]
)

prompt2 = PromptTemplate(
    template="Generate 5 Quiz questions and answers for the following: {text} \n",
    input_variables=["text"]
)

prompt3 = PromptTemplate(
    template="Merge the provided notes and quiz into a single document \n Notes: {notes} Quiz: {quiz}",
    input_variables=["notes", "quiz"]
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    "notes": prompt1 | model1 | parser,
    "quiz": prompt2 | model | parser
})

merge_chain = prompt3 | model1 | parser 

final_chain = parallel_chain | merge_chain

result = final_chain.invoke(text)

print(result)